# EmpkinS Mikro Data Loader

In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np
import pingouin as pg

import librosa as lr

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp
from biopsykit.signals.ecg import EcgProcessor

import empkins_io as eio
import empkins_micro as emic
from empkins_micro.preprocessing.imu.sensor_alignment import align_dataset_to_gravity, GRAV_VEC

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

figsize = (10, 5)

plt.close("all")
plt.rcParams['figure.figsize'] = figsize
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

pg.options['round'] = 3

palette

In [ ]:
subject_id = "VP_02"
condition = "stress"
study_part = "pre"

study_part_dict = {
    "pre": 0,
    "mist": 1,
    "post": 2
}

outlier_correction = ["statistical_rr", "statistical_rr_diff", "artifact", "physiological"]

In [ ]:
base_path = Path("../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy")
data_path = base_path.joinpath("data_per_subject/{}/{}".format(subject_id, condition))

ecg_path = data_path.joinpath("ecg/raw")
radar_path = data_path.joinpath("radar/raw")
video_path = data_path.joinpath("video")
timelog_path = data_path.joinpath("time_log/processed")

In [ ]:
ecg_files = list(sorted(ecg_path.glob("*.bin")))
radar_files = list(sorted(radar_path.glob("*.mat")))
video_files = list(sorted(video_path.glob("*.mp4")))

ecg_file = ecg_files[study_part_dict[study_part]]
radar_file = radar_files[study_part_dict[study_part]]
video_file = video_files[3]

timelog_file = timelog_path.joinpath("time_log_{}.csv".format(subject_id))
print("{}\t {}\t {}".format(ecg_file.name, radar_file.name, video_file.name))

## Load NilsPod and Radar Data

In [ ]:
ecg, fs_ecg = bp.io.nilspod.load_dataset_nilspod(ecg_file)
radar, fs_radar = eio.sensors.a04.load_data(radar_file, datastreams=["hr"])
radar = radar["hr"]

In [ ]:
ecg, radar = eio.sensors.utils.cut_data_to_overlap(ecg, radar)

#ecg["ecg"]= -1 * ecg["ecg"]

In [ ]:
time_intervals = bp.io.load_time_log(timelog_file, continuous_time=False)
time_intervals = time_intervals.filter(like=study_part)

time_intervals

In [ ]:
ep = EcgProcessor(data=ecg, sampling_rate=fs_ecg, time_intervals=time_intervals)
ep.ecg_process(outlier_correction=outlier_correction)
ecg_dict = ep.rpeaks

In [ ]:
rp = eio.processing.a04.A04Processor(radar, time_intervals=time_intervals)
rp.process(outlier_correction=outlier_correction)

radar_dict = rp.rpeaks

In [ ]:
key = "pre_baseline"

hr_ecg = ecg_dict[key]
hr_radar = radar_dict[key]
imu_data = ecg.loc[hr_ecg.index[0]:hr_ecg.index[-1]]

### Overview Plot

In [ ]:
fig, axs = bp.signals.ecg.plotting.ecg_plot(ep, key=key, figsize=figsize)

## Chest IMU Data

In [ ]:
acc_data_align = align_dataset_to_gravity(imu_data, sampling_rate_hz=fs_ecg).filter(like="acc")

# remove gravity
acc_data_align["acc_z"] -= np.sum(GRAV_VEC)
acc_energy_align = pd.DataFrame((acc_data_align**2).sum(axis=1), columns=["acc_energy"])

In [ ]:
fig, axs = plt.subplots(ncols=2)
acc_data_align.rolling(100).mean().plot(ax=axs[0], title="3-axis Acceleration")
acc_energy_align.rolling(100).mean().plot(ax=axs[1], title="Acceleration Energy")

## Audio Data

In [ ]:
audio_data, fs_audio = lr.load(video_file)
ds = 100
fs_downsample = fs_audio / ds
audio_downsample = audio_data[::ds]

In [ ]:
audio_downsample = pd.DataFrame(audio_downsample, index=hr_ecg.index[0] + pd.TimedeltaIndex(np.arange(0, len(audio_downsample)) / fs_downsample, unit='s'))
audio_downsample = np.abs(audio_downsample)
audio_downsample = audio_downsample.loc[hr_ecg.index[0]:hr_ecg.index[-1]]

In [ ]:
acc_roll = acc_energy_align.rolling(100).mean()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
fig, ax = eio.processing.a04.plotting.hr_plot_ecg_radar(hr_ecg=hr_ecg, hr_radar=hr_radar, plot_radar_quality=True, plot_outlier=False, plot_mean=False, ax=ax)

ylims = ax.get_ylim()

ymax = (ylims[1] - ylims[0]) * 0.5 + ylims[0]
ylims = [ylims[0], ymax]

acc_roll_rescaled = (ylims[1] - ylims[0]) * (acc_roll - acc_roll.min()) / (acc_roll.max() - acc_roll.min()) + ylims[0]
audio_rescaled = (ylims[1] - ylims[0]) * (audio_downsample - audio_downsample.min()) / (audio_downsample.max() - audio_downsample.min()) + ylims[0]

h = ax.plot(acc_roll_rescaled, color=bp.colors.fau_color("tech"), alpha=0.3)
l = ax.legend(h, ["Acc. Energy"], loc="lower left", fontsize="small")
ax.add_artist(l)
h = ax.plot(audio_rescaled, color=bp.colors.fau_color("phil"), alpha=0.3)
l = ax.legend(h, ["Audio Signal"], loc="lower right", fontsize="small")
ax.add_artist(l)